# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
output_data_file = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(output_data_file)
weather_data.head()


,city_id,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,progreso,75,MX,1595296133,94,21.28,-89.67,82.00,9.80
1,1,roald,100,NO,1595296189,87,62.58,6.13,51.80,11.41
2,2,jamestown,1,US,1595296033,60,42.10,-79.24,71.60,8.05
3,3,puerto ayora,99,EC,1595295716,87,-0.74,-90.35,69.01,8.01
4,4,new norfolk,92,AU,1595296014,75,-42.78,147.06,55.00,5.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

In [27]:
# Plot Heatmap
heat_fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

# Add layer
heat_fig.add_layer(heat_layer)

# Display figure
heat_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# weather conditions:
min_temp = weather_data[weather_data["Max Temp"]>50]
max_temp = min_temp[min_temp["Max Temp"]<80]
wind_speed = max_temp[max_temp["Wind Speed"]<7]
cloudiness = wind_speed[wind_speed["Cloudiness"]<75]
my_vacation = cloudiness[cloudiness["Humidity"]<75].reset_index(drop=True)
my_vacation

,city_id,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,5,ribeira grande,24,PT,1595296041,69,38.52,-28.70,69.17,2.04
1,15,tilichiki,46,RU,1595296120,62,60.47,166.10,58.59,5.53
2,33,lagoa,0,PT,1595296060,70,39.05,-27.98,68.50,3.78
3,45,broome,1,US,1595296101,53,42.25,-75.83,75.20,5.82
4,46,zelenoborsk,0,RU,1595296199,63,61.47,64.03,59.92,4.29
5,61,touho,16,NC,1595296203,57,-20.78,165.23,70.52,5.28
6,64,east london,0,ZA,1595296049,43,-33.02,27.91,61.34,5.21
7,87,narsaq,62,GL,1595296069,50,60.92,-46.05,51.80,5.82
8,95,goshen,20,US,1595295927,64,41.58,-85.83,75.20,6.93
9,101,altamira,0,BR,1595296212,71,-3.20,-52.21,74.91,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
my_vacation['Hotel Name'] = ""
hotel_df = pd.DataFrame(my_vacation)
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}

In [35]:
for index, row in hotel_df.iterrows():
    
    params["location"] =  f'{row["Lat"]},{row["Lng"]}'

    # get restaurant type from df
    hotel_location = row['City']


    # assemble url and make API request
    print(f"Retrieving Results for: {hotel_location}.")
    response = requests.get(base_url, params=params).json()
    print(response)
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['Name']

                  
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for: ribeira grande.
{'html_attributions': [], 'next_page_token': 'CqQCHgEAAIbNhznuW1drlDr_WEZf4zPUO5OBdVPWyG7He2x2jp2QRj660ZdG0DjYE9q70N4uqxklTU4D9NE6oxQNFTuQesqM0wVPVbV2a5OnO4Zo4EIIdOs5VbFswDuFkE5iLPYuMzYVuuJ8vyJOX-Jf06H9pUvAB94SwT3fJitTORWY308rZ-gReO7al46SKajRJhtjrK1Gpsjd34awBgrDng7V0z5YYvPT49Zm__c1lOYBXdrDMDve4hHk2_wLF49Jdm8_oQNuKMyAmUY9Yn83o43KopbDoiKdqVvXl__90_k0Vi9plSg4jh91IAQKjK-MOp1bmWt5--jRDsZioGjdpOmJ_XrgcDt9JHwI4UDA7p3ArYEiH6yFykzxT2DbmidbmRZtlxIQIOZjReby6zCy_XTaqK_A5hoUM060QN5K8bf2iaMLX_bkBS4MvMI', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 38.5222086, 'lng': -28.7105125}, 'viewport': {'northeast': {'lat': 38.5236150302915, 'lng': -28.7091731697085}, 'southwest': {'lat': 38.5209170697085, 'lng': -28.7118711302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '3917ce88f76ca5933eb917f2efdb01beb901e53b', 'name': 'Quinta Das Buganvílias', 'photos': [{'height': 4032, 'html_attributio

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
------------
Retrieving Results for: lagoa.
{'html_attributions': [], 'next_page_token': 'CqQCHgEAAAuObOytrE2a0KQ962t9LJ0i1XJsr2XdPq0nCW_IK24LMaNiE6WsIi500aFMJZYWnikniNvjSkt_f9rYyEOUqyKg7gmu9hgz_muYoc4XE4gRttbffXPC3OdobypxCw-ZMClRSDEH24T7EPtbVz9_oWcRMlmE0emR3Waxe8t6FhuF8OzKpl6pHszXeR2L7w2vgEt1-V2MgDHn3Gw7PYv93Blp82XLy62Ylur5G-zIzJAHv9Z77K-OPIimK9Ym8sb1VS9UTCu6BIfNcOt6WSHAaDPpZjTRVGg4cWiJYymdYkqQyCjxu6_aflDOgtQT_uWB8AtwjTb83KiyR4Bg0CL9ReENUAp2dEulFQKsaxnlUR9FYksLLspT1eOCeJF9JFEdsxIQLtUqGzxA8EII75N1DG7FahoUe_mnz17H1KN26qrpwP4QrLZuqaA', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 39.051463, 'lng': -27.978289}, 'viewport': {'northeast': {'lat': 39.05277468029151, 'lng': -27.9768480697085}, 'southwest': {'lat': 39.05007671970851, 'lng': -27.9795460302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/camping-71.png', 'id': 'ce6e716730c4e338

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 42.24626659999999, 'lng': -75.8301809}, 'viewport': {'northeast': {'lat': 42.2476469802915, 'lng': -75.8288355697085}, 'southwest': {'lat': 42.2449490197085, 'lng': -75.83153353029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/camping-71.png', 'id': '3a3d6564f9cc50ea1bf6d0c5bc789402b2cdeee6', 'name': 'Salisbury Chenango Shores', 'photos': [{'height': 3072, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117500496307060763037">Krista Haynes</a>'], 'photo_reference': 'CmRaAAAAjHkUA2PMJcykMDoaYuwgsjQX1gigHP7_2Q4F7F4NZVfgbNsLNEQrqevSbRH3jAIkMwxhAqlNQHtQCwM1zJHKdhAbJkdOAJAILUarnpqA4IUiKP9F_3v-HH5_8BIWWOiLEhBwA0rdwKPAGCkpNPMXg4b2GhQvbWDrxvmcNG_dDtuwzEdXX4_kNA', 'width': 1536}], 'place_id': 'ChIJ08V8eyL52okR-De2cH-4Gng', 'plus_code': {'compound_code': '65W9+GW Chenango Forks, NY, USA', 'global_code': '87J665W9+GW'}, 'rating': 4.1, 'reference': 'ChI

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
------------
Retrieving Results for: touho.
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.77509809999999, 'lng': 165.2378036}, 'viewport': {'northeast': {'lat': -20.7738367197085, 'lng': 165.2391549302915}, 'southwest': {'lat': -20.7765346802915, 'lng': 165.2364569697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '63d1096e3f0918fd87ce0adddadcd8b59a736978', 'name': 'Auberge Timay Evasion', 'place_id': 'ChIJ9QCoZIQWnG4RBiKhtFCv3ec', 'plus_code': {'compound_code': '66FQ+X4 Touho, New Caledonia', 'global_code': '5VF766FQ+X4'}, 'rating': 4.3, 'reference': 'ChIJ9QCoZIQWnG4RBiKhtFCv3ec', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 3, 'vicinity': 'RPN3, Touho'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -20.

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 60.91603109999999, 'lng': -46.04646919999999}, 'viewport': {'northeast': {'lat': 60.9174545302915, 'lng': -46.0449565197085}, 'southwest': {'lat': 60.91475656970849, 'lng': -46.04765448029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '405f4cc0393e8d816fc7c47882f162f40ecc56b5', 'name': 'Hotel Perlen', 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115754471240921168673">Herluf Kvist</a>'], 'photo_reference': 'CmRaAAAA32RYjh3XEAphDWvUqSBZKvUEcpjUGDcL3kj-eh9Zabt3zdULWp4rqDaV3mkcQE7Z3xqpm2h8IPlhudoq6e_yUdTNfKTVOzWpv5sUEGRDPmCpVodN4A1Vj05AULLtJ4ZiEhC04mQVQ-Y0vy65ongyckwSGhTUiUaGo2fijHeHZh2VE8Bg5tAyNQ', 'width': 5312}], 'place_id': 'ChIJHyxXR27Vq04Rp9IVhVQcBqs', 'plus_code': {'compound_code': 'WX83+CC Narsaq, Greenland', 'global_code': '98GMWX83+CC'}, 'rating': 3, 'reference': 'ChIJHyxXR27Vq04Rp

...

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
------------
Retrieving Results for: ilhabela.
{'html_attributions': [], 'next_page_token': 'CqQCHgEAAPj3U6rvxoHlroJ5Yyikf233M_bSh3-ZxLBkv6ppez_a5rBfuX23UKlRFxfOniG_HOM1J5YLKpXOsm87BSnCKBT13sfWRVK9XfQv_58YNOEiW_XBcvV1DrrGSl6-7Seaz_9Efz_ViDbyuu2XFsq3yGEeQh5lMI_nbWDqEevPnm8daJ0lLotrjPkzcnGNNmco2RPgpPDvo4Llp1bMTM_D4soOyd5oyLytp06jKKiQ-VtCQ0HM0IO2w7ssV7DsIqxddPv3Aavl_NOocxtzdUnSSvpr5x92R0D_v6O97fFxU3ZVvaqaJY7G8pQEq1QYHoYB4UW02Kvcg49PKoLmPoLbIc7jgF3xxiEN4onVVHANwww4Cm_CBFTMoUXI_Qsef6mEnxIQnBeAcN7Wbx9j2mBTNULgMRoUV8rdbP46LXMGPm3sF9qDVRBKpTo', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.7793844, 'lng': -45.3581169}, 'viewport': {'northeast': {'lat': -23.7779915197085, 'lng': -45.3569317197085}, 'southwest': {'lat': -23.7806894802915, 'lng': -45.3596296802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '579327c5df

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -3.4092007, 'lng': 29.1415477}, 'viewport': {'northeast': {'lat': -3.407697519708498, 'lng': 29.1429863302915}, 'southwest': {'lat': -3.410395480291502, 'lng': 29.1402883697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'c698fd2addfa7f89a72da1a66ad86ad614cdbf1a', 'name': 'Auberge Eden City', 'photos': [{'height': 2822, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113998645472108047252">victoire kitu</a>'], 'photo_reference': 'CmRaAAAAwW_PkW7rDe5UTnM-t5Y1-0q5uw4fbyomIqlfLBXOIQqvOwn8Z8yvgc7sLtitU4RBL2YSR4JSWto9o3GHA_vipS9qXiQDa50FMDvprYev2RoATh6l8fsDCSJ4Jj1a7fscEhAjVVY6qnZMVainLxIPzhr_GhSTYcT9WrIO29GVuSE1SznBmOGgDA', 'width': 1708}], 'place_id': 'ChIJ4SYO2gCOwRkRnVg1JS6I4Ew', 'plus_code': {'compound_code': 'H4RR+8J Uvira, Democratic Republic of the Congo', 'global_code': '6G8FH4RR+8J'}, 'rating': 2.6, 'reference': 'Ch

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
------------
Retrieving Results for: olean.
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 42.0772605, 'lng': -78.4301808}, 'viewport': {'northeast': {'lat': 42.0786242802915, 'lng': -78.42868746970849}, 'southwest': {'lat': 42.0759263197085, 'lng': -78.4313854302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': '6c0e42f389661bb12a5e643809cb3a60b246dab1', 'name': 'Old Library Events', 'photos': [{'height': 5312, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105056137883554247803">Cleo Weiss</a>'], 'photo_reference': 'CmRaAAAA_Y9X5cZ8hk7dkiAqXR9Rspc-cDvrSH8Zrk8eWL6H0C1TXmMJAjDS5jl6OrJKpvY4N6BkSKVNxLj07nzLxxyIdxinaWerJfb5gPOqFl5NaS4P-RsVozhPpulah2a47dOYEhAaZ5e2KOi8dStVMF2Mst64GhQdQfeKAl55YNyKzekFfbELrdeLzA', 'width': 2988}], 'place_id': 'ChIJXQGG8v1w0okRWCIZfTlrUO0', '

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
------------
Retrieving Results for: alta floresta.
{'html_attributions': [], 'next_page_token': 'CqQCHgEAAKxnJE8pUj52W7zKgO4HmA6gnuC5FrjsjFoGSm-HwV9eDJIwLQ0xgAVIyEIUwKgX9Tc83yynNMkIjcja7Pe6jFPoCqd52TLymBoItjWfPHziILFDiDuzR7vOr9Riai4XTJt5xtxMqxLyoBZZ6xhspTnluXzb_QIGLbnY1yTtHUgSe1-PDISZMhtRdWEr-uJVzU_1886OZCmze0XEuWIsNWYhE28w2vaWYyOaI5PluQgfwfI08Ddmwf_mXK9ARuunJs8jtqqmTsJaZ8LC73XjsrUk7qqDuDzDeVmYkiUvY5q0j1Mkq7e36nYEhmZQRxM8l-hhU8OI1N_-X29FWY5tNHveUr46MrrmFOgeZUqa-5XeQU3FGTgLLsh5gXDWskJQXxIQUDIL2zxCxqDfDF_7O5nh3hoUZm1K1Two2Z4HRnfpzQoXfwv3_hw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -9.8798679, 'lng': -56.08805160000001}, 'viewport': {'northeast': {'lat': -9.878523269708497, 'lng': -56.0866710697085}, 'southwest': {'lat': -9.881221230291501, 'lng': -56.08936903029149}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 29.1263232, 'lng': 15.9557748}, 'viewport': {'northeast': {'lat': 29.1277753302915, 'lng': 15.9570922802915}, 'southwest': {'lat': 29.1250773697085, 'lng': 15.9543943197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'id': 'fca686aa0f8ab7efa70bafdf262a886b0549184f', 'name': 'الذوات للفيلوات', 'place_id': 'ChIJ4--QBcN5vhMRdEwwFbAw3Sk', 'plus_code': {'compound_code': '4XG4+G8 الخمسمائة، Libya', 'global_code': '7FXQ4XG4+G8'}, 'rating': 2.5, 'reference': 'ChIJ4--QBcN5vhMRdEwwFbAw3Sk', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 4, 'vicinity': 'شارع الخمسمائة'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 29.1247653, 'lng': 15.9579618}, 'viewport': {'northeast': {'lat': 29.1261574302915, 'lng': 15.9595051802915}, 'southwest': {'lat': 29.1234594697085, 'lng': 15.9568072197

In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_fig.add_layer(marker_layer)

# Display figure
heat_fig

Figure(layout=FigureLayout(height='420px'))